## The Page Rank Algorithm

In [90]:
import numpy as np 
from numpy import linalg as la
from scipy.sparse import dok_matrix

### Probem 1.
#### reading in a matrix file annd turning 'sinks' into rows of ones

In [83]:
def make_adj_mat(filename, N):
    A = np.zeros((N, N))
    with open(filename) as f:
        for line in f:
            try:
                i, j = line.strip().split()
                i = int(i)
                j = int(j)
                A[i, j] = 1   
            except ValueError:
                pass
            
        is_sink = A.sum(axis=1) == 0
        sinks = np.where(is_sink == True)
        A[sinks,:] = 1
    return dok_matrix(A)         

In [85]:
A = make_adj_mat(r'matrix.txt', 8)
print(A.toarray())

[[ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  1.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]]


### Problem 2.
#### compute the K matrix for a given adj matrix A

In [107]:
def make_K(A):
    A = A.toarray()
    Outs = A.sum(axis=1)
    D = np.diag(Outs)
    print(Outs.shape)
    
    K = A.T / Outs
    
    return dok_matrix(K)
    
    

In [110]:
K = make_K(A)
print(K)

(8,)
  (1, 2)	0.125
  (3, 2)	0.125
  (6, 4)	0.333333333333
  (0, 4)	0.333333333333
  (5, 4)	0.333333333333
  (0, 7)	1.0
  (6, 2)	0.125
  (0, 5)	0.5
  (4, 2)	0.125
  (0, 3)	0.333333333333
  (7, 2)	0.125
  (0, 1)	1.0
  (7, 0)	1.0
  (6, 3)	0.333333333333
  (0, 6)	1.0
  (6, 5)	0.5
  (2, 2)	0.125
  (2, 3)	0.333333333333
  (5, 2)	0.125
  (0, 2)	0.125


### Probllem 3.
#### compute the Page rank numerically

In [144]:
def get_pagerank_num(A, N=None, d=0.85, tol=1e-5):
    if N is None:
        N = np.min(A.shape)
    A_upper = A[:N,:N]
    
    K = make_K(A)
    K = K.toarray()
    
    p_0 = np.random.random(N).reshape((N,1))
    const = ((1 -d) / N ) * np.ones((N, 1))
    pdist = 1
    while pdist > tol:
        p_t = d * K @ p_0 + const
        pdist = la.norm(p_t - p_0)
        p_0 = np.copy(p_t)
    
    return p_0

In [145]:
p_star = get_pagerank_num(A)
p_star

(8,)


array([[ 0.43870408],
       [ 0.02171029],
       [ 0.02786154],
       [ 0.02171029],
       [ 0.02171029],
       [ 0.02786154],
       [ 0.04585394],
       [ 0.39460461]])

### Problem 4.
#### compute Page rank using eigenvalues


In [167]:
def get_pagerank_eig(A, N=None, d=0.85):
    if N is None:
        N = np.min(A.shape)
    A_upper = A[:N,:N]
    
    K = make_K(A)
    K = K.toarray()
    
    p_0 = np.random.random(N).reshape((N,1))
    
    const = ((1 -d) / N ) * np.ones((N, 1))
    B = (d * K + const)
    
    w, v = la.eig(B)
    
    p_t = v[0]
    
    return p_t 

In [169]:
b = get_pagerank_eig(A)
b

(8,)


array([ -7.38129111e-01,  -7.07106781e-01,   2.36517595e-01,
        -9.45845122e-17,  -1.29826927e-01,  -2.77839441e-09,
         2.77839403e-09,  -1.86179804e-16])